In [1]:
import pyodbc
from datetime import datetime,timedelta
import win32com.client as win32
 
# String de conexão
connection_string = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER=;"
    f"DATABASE=;"
    f"UID=;"
    f"PWD="
)

def EncaminhaEmails(CodigoCarteira, ValidaEnvio,idDaily):
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()

    query_operacao = """execute geral.EmailDailyToPy ?, ?, ?"""
    
    try:
        cursor.execute(query_operacao, CodigoCarteira, ValidaEnvio,idDaily)
        conn.commit()  #
        print("Mail queued.", CodigoCarteira)

    except Exception as e:
        print(f"Erro: {e}")

    finally:
        cursor.close()
        conn.close()

def AtualizaMacro(CodigoCarteira, Banco, Carteira, Filial):
    # Data de hoje
    data_hoje = datetime.now()


    if(data_hoje.day == 1):
        data_hoje = data_hoje - timedelta(days=1)
        
    # Primeira data do mês
    primeira_data_do_mes = data_hoje.replace(day=1)

    # Formatar as datas para o formato aaaa-mm-dd
    data_hoje_formatada = data_hoje.strftime("%Y-%m-%d")
    primeira_data_do_mes_formatada = primeira_data_do_mes.strftime("%Y-%m-%d")
    
    arrayParaVisualizarMelhor = [CodigoCarteira, Banco, Carteira, Filial]
    
    # Caminho do arquivo Excel
    caminho_excel = rf"\\172.17.1.115\TEMP\\DailysToPy\Dailys\{Filial}\Daily_{CodigoCarteira}_{Banco}_{Carteira}.xlsb"

    excel = None
    try:
        # Inicia uma instância do Excel
        excel = win32.Dispatch("Excel.Application")
        excel.Visible = False

        # Abre o arquivo Excel
        workbook = excel.Workbooks.Open(Filename=caminho_excel)
        
        # Habilita as macros
        excel.AutomationSecurity = 1  # Habilita todas as macros
        
        # Atualiza as células
        sheet = workbook.Sheets('Aux_SQL')
        sheet.Range("G2").Value = primeira_data_do_mes_formatada
        sheet.Range("H2").Value = data_hoje_formatada
        
        # Executa a macro
        excel.Application.Run('Atualiza_Tudo.Atu_tudo')
        
        # Salva e fecha o arquivo
        workbook.Save()
        workbook.Close(SaveChanges=1)

    except Exception as e:
        excel.Application.Quit()
        del excel
    finally:
        excel.Application.Quit()
        del excel
        print(f"Processo concluído para {arrayParaVisualizarMelhor}.")




def ConsultaValida_ToAtualizaMacro():
    # Data de hoje
    data_hoje = datetime.now()


    if(data_hoje.day == 1):
        data_hoje = data_hoje - timedelta(days=1)

    # Formatar as datas para o formato aaaa-mm-dd
    data_hoje_formatada = data_hoje.strftime("%Y-%m-%d")
    # Tentar se conectar e processar
    try:
        conn = pyodbc.connect(connection_string)
        cursor = conn.cursor()

        # Consulta SQL
        query = """  
                    select	distinct
                        a.CodigoCarteira,
                        b.Banco,
                        b.Carteira,
                        b.Filial,
                        (case when a.msg = 'Enviar' then 1 else 0 end) msg,
                        --1 msg,
                        b.IdDaily
                                        
                    from dbdatastorehouse.dbo.MapaDailyPesoToPy a with(nolock)
                    join dbdatastorehouse.dbo.dp_dailytopy b on a.CodigoCarteira = b.GrupoCliente
                    where convert(date,a.DataAtualizacao) = ?
                                                                                                                                            
                    order by Banco asc, Filial asc
                """

        # Executar a consulta e armazenar os resultados
        cursor.execute(query, data_hoje_formatada)
        arrayCdCli = cursor.fetchall()
        
        # Fechar conexão
        cursor.close()
        conn.close()
        
        if arrayCdCli:
            for row in arrayCdCli:
                CodigoCarteira, Banco, Carteira, Filial, Msg, idDaily = row

                # Tentar rodar a macro para cada resultado
                try:
                    if (Msg == 1):
                        #print('Comecei a atualizar o arquivo!')
                        AtualizaMacro(CodigoCarteira, Banco, Carteira, Filial)
                        EncaminhaEmails(CodigoCarteira, Msg, idDaily)
                    else:
                        EncaminhaEmails(CodigoCarteira, Msg, idDaily)
                except Exception as e:
                    print(f"Erro em: {row}: {e}")
        else:
            print('Nenhum resultado encontrado.')
    except Exception as e:
        print(f"Erro ao conectar ou executar a consulta: {e}")

# Executar a função apenas uma vez
ConsultaValida_ToAtualizaMacro()


Processo concluído para [151, 'Bradesco', 'BBF', 'Marechal'].
Mail queued. 151
Processo concluído para [155, 'Bradesco', 'VEICULOS', 'Marechal'].
Mail queued. 155
Processo concluído para [575, 'Bradesco', 'ATIVOS', 'Marechal'].
Mail queued. 575
Processo concluído para [667, 'Bradesco', 'LOSANGO', 'Marechal'].
Mail queued. 667
Processo concluído para [853, 'Bradesco', 'RECOVERY', 'Marechal'].
Mail queued. 853
Processo concluído para [903, 'Bradesco', 'SAUDE', 'Marechal'].
Mail queued. 903
Processo concluído para [911, 'Bradesco', 'COMERCIAL', 'Marechal'].
Mail queued. 911
Processo concluído para [913, 'Bradesco', 'CI', 'Marechal'].
Mail queued. 913
Processo concluído para [915, 'Bradesco', 'CI_DEJUR', 'Marechal'].
Mail queued. 915
Processo concluído para [32, 'BV', 'REMANESCENTE', 'Aviacao'].
Mail queued. 32
Processo concluído para [135, 'BV', 'MASSIFICADO', 'Aviacao'].
Mail queued. 135
Processo concluído para [137, 'BV', 'CARTOES', 'Aviacao'].
Mail queued. 137
Processo concluído para [

In [5]:
data_hoje = datetime.now()


if(data_hoje.day == 1):
    data_hoje = data_hoje - timedelta(days=1)

print(data_hoje)

2025-01-07 06:43:51.910460
